In [78]:
#import statements
import pandas as pd
from bs4 import BeautifulSoup
import urllib3

Motivation for Performing The Analysis
=====================================
I'm looking to analyze the Android market determine what are the most popular types of apps,
looking at the different categories of apps and how they rank,
and what applications and types are users rating the highest
#### Main Data Source:
[Kaggle - Google Play Store Apps](#https://www.kaggle.com/lava18/google-play-store-apps)
Data has been included in this Github Repository

In [79]:
googlereviews = pd.read_csv('googleplaystore.csv')
googlesent = pd.read_csv('googleplaystore_user_reviews.csv')

#### Web Scraping Data Source
[App Annie](https://www.appannie.com/en/apps/google-play/top/)
Here the data is web scrapped

In [80]:
http = urllib3.PoolManager()
response = http.request('GET',
                            'https://www.appannie.com/en/apps/google-play/top/')
soup = BeautifulSoup(response.data)
table = soup.find_all('div', attrs={'class': 'textContainer_ssmtv3'})
print(table)

[<div class="textContainer_ssmtv3"><p class="n_1jv8115-o_O-Content_1xn1r0f-o_O-c0f2346_4s8r2l-o_O-text_7y41qf-o_O-item_9xo99e">GasBuddy: Find and Pay for Cheap Gas and Fuel</p><p class="n_1jv8115-o_O-Small_7u8o3i-o_O-c8790a2_1bcd4jz-o_O-item_9xo99e">GasBuddy</p></div>, <div class="textContainer_ssmtv3"><p class="n_1jv8115-o_O-Content_1xn1r0f-o_O-c0f2346_4s8r2l-o_O-text_7y41qf-o_O-item_9xo99e">Crypto.com - Buy Bitcoin Now</p><p class="n_1jv8115-o_O-Small_7u8o3i-o_O-c8790a2_1bcd4jz-o_O-item_9xo99e">Crypto Technology Holdings Limited</p></div>, <div class="textContainer_ssmtv3"><p class="n_1jv8115-o_O-Content_1xn1r0f-o_O-c0f2346_4s8r2l-o_O-text_7y41qf-o_O-item_9xo99e">Coinbase – Buy &amp; Sell Bitcoin. Crypto Wallet</p><p class="n_1jv8115-o_O-Small_7u8o3i-o_O-c8790a2_1bcd4jz-o_O-item_9xo99e">Coinbase Android</p></div>, <div class="textContainer_ssmtv3"><p class="n_1jv8115-o_O-Content_1xn1r0f-o_O-c0f2346_4s8r2l-o_O-text_7y41qf-o_O-item_9xo99e">Slice It All!</p><p class="n_1jv8115-o_O-Small

###### Cleaning the data
The data that is scrapped from the web, there are three tables on the web page. Table for paid, grossing and free.
The data is stripped down to only find teh table with the applications names, ranking and type.

In [81]:
rank=1
ranked = []
track = 0
for rows in table:
    appname = rows.find('p', attrs={'class': 'n_1jv8115-o_O-Content_1xn1r0f-o_O-c0f2346_4s8r2l-o_O-text_7y41qf-o_O-item_9xo99e'}).get_text().strip()
    appname
    if track < 50:
        type = 'free'
    elif track < 100:
        type = 'paid'
    else:
        type = 'grossing'

    ranked.append([rank, appname, type])
    if rank ==50 or rank == 100:
        rank = 0
    rank += 1
    track += 1

rankeddf = pd.DataFrame(ranked, columns=["ranking", "name", "type"])
rankeddf.head()

,ranking,name,type
0,1,GasBuddy: Find and Pay for Cheap Gas and Fuel,free
1,2,Crypto.com - Buy Bitcoin Now,free
2,3,Coinbase – Buy & Sell Bitcoin. Crypto Wallet,free
3,4,Slice It All!,free
4,5,Catwalk Beauty,free
...,...,...,...
145,46,Huuuge Casino™ Free Slots & Best Slot Machines...,grossing
146,47,Vegas Slots - DoubleDown Casino,grossing
147,48,Lotsa Slots - Free Vegas Casino Slot Machines,grossing
148,49,King of Avalon: Dominion,grossing
